# Практическое задание 4

# Распознавание именованных сущностей из Twitter с помощью LSTM

## курс "Математические методы анализа текстов"


### ФИО: <впишите>

## Введение

### Постановка задачи

В этом задании вы будете использовать рекуррентные нейронные сети для решения проблемы распознавания именованных сущностей (NER). Примерами именованных сущностей являются имена людей, названия организаций, адреса и т.д. В этом задании вы будете работать с данными twitter.

Например, вы хотите извлечь имена и названия организаций. Тогда для текста

    Yan Goodfellow works for Google Brain

модель должна извлечь следующую последовательность:

    B-PER I-PER    O     O   B-ORG  I-ORG

где префиксы *B-* и *I-* означают начало и конец именованной сущности, *O* означает слово без тега. Такая префиксная система введена, чтобы различать последовательные именованные сущности одного типа.

Решение этого задания будет основано на нейронных сетях, а именно на Bi-Directional Long Short-Term Memory Networks (BiLSTMs). В базовой части задания вам также нужно будет улучшить модель при помощи необучаемого пост-процессинга, основанного на алгоритме Витерби и графической модели CRF. В бонусной части вам будет предложено полноценно использовать связку BiLSTM и CRF, обучая обе модели одновременно.

### Библиотеки

Для этого задания вам понадобятся следующие библиотеки:
 - [Pytorch](https://pytorch.org/).
 - [Numpy](http://www.numpy.org).

### Данные

Все данные содержатся в папке `./data`: `./data/train.txt`, `./data/validation.txt`, `./data/test.txt`.

Скачать архив можно здесь: [ссылка на google диск](https://drive.google.com/open?id=1s1rFOFMZTBqtJuQDcIvW-8djA78iUDcx)

In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim

## Часть 1. Подготовка данных (2 балла)

#### Баллы за эту часть можно получить только при успешном выполнении части 2.

### Загрузка данных

Мы будем работать с данными, которые содержат твиты с тегами именованных сущностей. Каждая строка файла содержит пару токен (слово или пунктуация) и тег, разделенные пробелом. Различные твиты разделены пустой строкой.

Функция *read_data* считывает корпус из *file_path* и возвращает два списка списков: один со списками токенов по твитам и один со списками соответствующих токенам тегов. Также она заменяет все ники (токены, которые начинаются на символ *@*) на токен `<USR>` и url-ы (токены, которые начинаются на *http://* или *https://*) на токен `<URL>`.

**<font color='red'>Задание. Реализуйте функцию read_data.</font>**

In [180]:
def read_data(file_path):
    tokens = []
    tags = []

    ######################################
    ######### YOUR CODE HERE #############
    ######################################
    tokens_twit = []
    tags_twit = []
    with open(file_path, 'r') as file:
        for pair in file:
            if len(pair.strip()) == 0:
                tokens.append(tokens_twit)
                tags.append(tags_twit)
                
                tokens_twit = []
                tags_twit = []
            else:
                token, tag = pair.strip().split()
                tokens_twit.append(token)
                tags_twit.append(tag)
    return tokens, tags

Теперь мы можем загрузить 3 части данных:
 - *train* для тренировки модели;
 - *validation* для валидации и подбора гиперпараметров;
 - *test* для финального тестирования.

In [181]:
train_sentences, train_tags = read_data('data/train.txt')
val_sentences, val_tags = read_data('data/validation.txt')
test_sentences, test_tags = read_data('data/test.txt')

In [144]:
train_sentences[0], train_tags[0]

(['RT',
  '@TheValarium',
  ':',
  'Online',
  'ticket',
  'sales',
  'for',
  'Ghostland',
  'Observatory',
  'extended',
  'until',
  '6',
  'PM',
  'EST',
  'due',
  'to',
  'high',
  'demand',
  '.',
  'Get',
  'them',
  'before',
  'they',
  'sell',
  'out',
  '...'],
 ['O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'B-musicartist',
  'I-musicartist',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O',
  'O'])

Всегда полезно знать, с какими данными вы работаете. Выведем небольшую часть.

In [145]:
for i in range(3):
    for token, one_tag in zip(train_sentences[i], train_tags[i]):
        print('%s\t%s' % (token, one_tag))
    print()

RT	O
@TheValarium	O
:	O
Online	O
ticket	O
sales	O
for	O
Ghostland	B-musicartist
Observatory	I-musicartist
extended	O
until	O
6	O
PM	O
EST	O
due	O
to	O
high	O
demand	O
.	O
Get	O
them	O
before	O
they	O
sell	O
out	O
...	O

Apple	B-product
MacBook	I-product
Pro	I-product
A1278	I-product
13.3	I-product
"	I-product
Laptop	I-product
-	I-product
MD101LL/A	I-product
(	O
June	O
,	O
2012	O
)	O
-	O
Full	O
read	O
by	O
eBay	B-company
http://t.co/2zgQ99nmuf	O
http://t.co/eQmogqqABK	O

Happy	O
Birthday	O
@AshForeverAshey	O
!	O
May	O
Allah	B-person
s.w.t	O
bless	O
you	O
with	O
goodness	O
and	O
happiness	O
.	O



### Подготовка словарей

Чтобы обучать нейронную сеть, мы будем использовать два отображения.

- {token}$\to${token id}: устанавливает соответствие между токеном и строкой в embedding матрице;
- {tag}$\to${tag id}: one hot encoding тегов.


Теперь вам необходимо реализовать функцию *build_dict*, которая должна возвращать словарь {token or tag}$\to${index} и контейнер, задающий обратное отображение.

**<font color='red'>Задание. Реализуйте функцию build_dict.</font>**

In [146]:
def build_dict(entities, special_entities):
    """
    Args:
        entities: a list of lists of tokens
        special_entities: a list of special tokens

    Returns:
        entity_to_idx : mapping to index
        idx_to_entity : mapping from index
    """
    entity_to_idx = dict()
    idx_to_entity = []

    # Create mappings from tokens to indices and vice versa
    # Add special tokens to dictionaries
    # The first special token must have index 0

    ######################################
    ######### YOUR CODE HERE #############
    ######################################
    
    for idx, special in enumerate(special_entities):
        entity_to_idx[special] = idx
        idx_to_entity.append(special)
    
    current_idx = len(special_entities)
    for entity_list in entities:
        for token in entity_list:
            if token not in entity_to_idx:
                entity_to_idx[token] = current_idx
                idx_to_entity.append(token)
                current_idx += 1

    return entity_to_idx, idx_to_entity

После реализации функции *build_dict* вы можете создать словари для токенов и тегов. В нашем случае специальными токенами будут:
 - `<UNK>` токен для обозначаения слов, которых нет в словаре;
 - `<PAD>` токен для дополнения предложений одного батча до одинаковой длины.

In [147]:
special_tokens = ['<UNK>', '<PAD>']
special_tags = []

# Create dictionaries
token_to_idx, idx_to_token = build_dict(train_sentences + val_sentences, special_tokens)
tag_to_idx, idx_to_tag = build_dict(train_tags, special_tags)

### Подготовка датасета и загрузчика

Обычно нейронные сети обучаются батчами. Это означает, что каждое обновление весов нейронной сети происходит на основе нескольких последовательностей. Технической деталью является необходимость дополнить все последовательности внутри батча до одной длины.

Для начала необходимо реализовать <<датасет>> для хранения ваших данных. Датасет должен наследоваться от стандартного pytorch класса `Dataset` и переопределять методы `__getitem__` и `__len__`. Метод `__getitem__` должен возвращать индексированную последовательность и её теги. Не забудьте про `<UNK>` токен для неизвестных слов!

**<font color='red'>Задание. Реализуйте класс TaggingDataset.</font>**

In [148]:
from torch.utils.data import Dataset, DataLoader


class TaggingDataset(Dataset):

    def __init__(self, sentences, tags, token_to_idx, tag_to_idx):
        """
        Args:
            sentences: a list of lists of tokens
            tags: a list of lists of corresponding tags
            token_to_idx: mapping from token to token indexes
            tag_to_idx: mapping from tag to tag indexes
        """
        super().__init__()
        ######################################
        ######### YOUR CODE HERE #############
        ######################################
        self.sentences = sentences
        self.tags = tags
        self.token_to_idx = token_to_idx
        self.tag_to_idx = tag_to_idx

    def __getitem__(self, idx):
        """
        Args:
            idx : int

        Returns:
            sentence_idx : torch.tensor of token indexes
            tag_idx : torch.tensor of tag indexes
        """
        ######################################
        ######### YOUR CODE HERE #############
        ######################################
        sentence = self.sentences[idx]
        tags = self.tags[idx]
        sentence_idx = []
        tag_idx = []
        for token, tag in zip(sentence, tags):
            if token in self.token_to_idx:
                sentence_idx.append(self.token_to_idx[token])
            else:
                sentence_idx.append(self.token_to_idx["<UNK>"])
            tag_idx.append(self.tag_to_idx[tag])

        sentence_idx = torch.tensor(sentence_idx, dtype=torch.int64)
        tag_idx = torch.tensor(tag_idx, dtype=torch.int64)
        
        return sentence_idx, tag_idx
        
        

    def __len__(self):
        ######################################
        ######### YOUR CODE HERE #############
        ######################################
        return len(self.sentences)


Для того, чтобы дополнять последовательности паддингом, будем использовать параметр `collate_fn` класса `DataLoader`. Принимая последовательность пар тензоров для предложений и тегов, необходимо дополнить все последовательности до последовательности максимальной длины в батче. Используйте специальные теги `<PAD>` и `O` для дополнения.

**<font color='red'>Задание. Реализуйте класс PaddingCollator.</font>**

In [149]:
from torch.nn.utils.rnn import pad_sequence


class PaddingCollator:
    def __init__(self,  pad_token_id, pad_tag_id, batch_first=True):
        self.pad_token_id = pad_token_id
        self.pad_tag_id = pad_tag_id
        self.batch_first = batch_first

    def __call__(self, batch):
        """
        Args:
            batch: list of tuples of torch.tensors

        Returns:
            new_sentences: torch.tensor
            new_tags: torch.tensor
                Both tensors have the same size
        """
        ######################################
        ######### YOUR CODE HERE #############
        ######################################
        sentences, tags = zip(*batch)

        new_sentences = pad_sequence(sentences, batch_first=self.batch_first, padding_value=self.pad_token_id)
        new_tags = pad_sequence(tags, batch_first=self.batch_first, padding_value=self.pad_tag_id)
        return new_sentences, new_tags

Теперь всё готово, чтобы задать DataLoader. Протестируйте на примере ниже, что всё работает правильно.

In [150]:
small_dataset = TaggingDataset(
    sentences=train_sentences[:7],
    tags=train_tags[:7],
    token_to_idx=token_to_idx,
    tag_to_idx=tag_to_idx,
)

small_loader = DataLoader(
    small_dataset,
    batch_size=3,
    shuffle=False,
    drop_last=False,
    collate_fn=PaddingCollator(
        pad_token_id=token_to_idx['<PAD>'],
        pad_tag_id=tag_to_idx['O'],
        batch_first=True,
    ),
)

batch_lengths = [3, 3, 1]
sequence_lengths = [26, 25, 8]
some_pad_tensor = torch.LongTensor([token_to_idx['<PAD>']] * 12)
some_outside_tensor = torch.LongTensor([[tag_to_idx['O']] * 12])

for i, (tokens_batch, tags_batch) in enumerate(small_loader):
    assert tokens_batch.dtype == torch.int64, 'tokens_batch is not LongTensor'
    assert tags_batch.dtype == torch.int64, 'tags_batch is not LongTensor'

    assert len(tokens_batch) == batch_lengths[i], 'wrong batch length'

    for one_token_sequence in tokens_batch:
        assert len(one_token_sequence) == sequence_lengths[i], 'wrong length of sequence in batch'

    if i == 0:
        assert torch.all(tokens_batch[2][-12:] == some_pad_tensor), "wrong padding"
        assert torch.all(tags_batch[2][-12:] == some_outside_tensor), "wrong O tag"

**<font color='red'>Задание. В ячейке ниже задайте датасеты и загрузчики для обучающих, валидационных и тестовых данных.</font>**

In [152]:
######################################
######### YOUR CODE HERE #############
######################################

train_dataset = TaggingDataset(
    sentences=train_sentences,
    tags=train_tags,
    token_to_idx=token_to_idx,
    tag_to_idx=tag_to_idx,
)

train_loader = DataLoader(
    train_dataset,
    batch_size=256,
    shuffle=True,
    drop_last=False,
    collate_fn=PaddingCollator(
        pad_token_id=token_to_idx['<PAD>'],
        pad_tag_id=tag_to_idx['O'],
        batch_first=True,
    ),
)

In [153]:
val_dataset = TaggingDataset(
    sentences=val_sentences,
    tags=val_tags,
    token_to_idx=token_to_idx,
    tag_to_idx=tag_to_idx,
)

val_loader = DataLoader(
    val_dataset,
    batch_size=256,
    shuffle=False,
    drop_last=False,
    collate_fn=PaddingCollator(
        pad_token_id=token_to_idx['<PAD>'],
        pad_tag_id=tag_to_idx['O'],
        batch_first=True,
    ),
)

In [154]:
test_dataset = TaggingDataset(
    sentences=test_sentences,
    tags=test_tags,
    token_to_idx=token_to_idx,
    tag_to_idx=tag_to_idx,
)

test_loader = DataLoader(
    test_dataset,
    batch_size=256,
    shuffle=False,
    drop_last=False,
    collate_fn=PaddingCollator(
        pad_token_id=token_to_idx['<PAD>'],
        pad_tag_id=tag_to_idx['O'],
        batch_first=True,
    ),
)

## Часть 2. BiLSTM-теггер (4 балла)

Определите архитектуру сети, используя библиотеку pytorch.

Ваша архитектура в этом пункте должна соответствовать стандартному теггеру (см. лекцию):
* Embedding слой на входе
* Двунаправленный LSTM слой для обработки последовательности
* Используйте dropout (заданный отдельно или встроенный в LSTM) для уменьшения переобучения
* Linear слой на выходе

Для обучения сети используйте поэлементную кросс-энтропийную функцию потерь.
**Обратите внимание**, что `<PAD>` токены не должны участвовать в подсчёте функции потерь. В качестве оптимизатора рекомендуется использовать Adam. Для получения значений предсказаний по выходам модели используйте функцию $\arg\max$.

**<font color='red'>Задание. Задайте архитектуру сети и требуемые методы.</font>**

In [156]:
class BiLSTMModel(torch.nn.Module):
    def __init__(
        self,
        vocabulary_size,
        tag_space_size,
        pad_token_idx,
        embedding_dim,
        lstm_hidden_size,
        dropout_zeroed_probability,
        device='cuda',
        max_norm=None,
    ):
        '''
        Defines neural network structure.

        architecture: input -> Embedding -> BiLSTM with Dropout -> Linear

        ----------
        Parameters

        vocabulary_size: int, number of words in vocabulary.
        tag_space_size: int, number of tags.
        pad_token_idx: int, index of padding character. Used for loss masking.
        embedding_dim: int, dimension of words' embeddings.
        lstm_hidden_size: int, number of hidden units in each LSTM cell
        dropout_zeroed_probability: float, dropout zeroed probability for Dropout layer.
        device: str, cpu or cuda:x
        '''
        ######################################
        ######### YOUR CODE HERE #############
        ######################################

        super(BiLSTMModel, self).__init__()
        
        self.embedding = nn.Embedding(
            num_embeddings=vocabulary_size,
            embedding_dim=embedding_dim,
            padding_idx=pad_token_idx,
            max_norm=max_norm,
        )

        self.lstm = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=lstm_hidden_size,
            batch_first=True,
            dropout=dropout_zeroed_probability,
            bidirectional=True
        )

        # self.dropout = nn.Dropout(p=dropout_zeroed_probability)
        
        self.linear = nn.Linear(
            in_features=lstm_hidden_size * 2,
            out_features=tag_space_size
        )
        self.device = device


    def forward(self, x_batch):
        '''
        Makes forward pass.

        ----------
        Parameters
        x_batch: torch.LongTensor with shape (number of samples in batch, number words in sentence).
        '''
        ######################################
        ######### YOUR CODE HERE #############
        ######################################
        x_embedded = self.embedding(x_batch)
        lstm_out, _ = self.lstm(x_embedded)
        return self.linear(lstm_out)

    def predict_for_batch(self, x_batch):
        '''
        Returns predictions for x_batch. Use argmax function.

        return type: torch.LongTensor
        return shape: (number of samples in batch, number words in sentence.

        ----------
        Parameters
        x_batch: torch.LongTensor with shape (number of samples in batch, number words in sentence).
        '''
        ######################################
        ######### YOUR CODE HERE #############
        ######################################
        x_batch = x_batch.to(self.device)
        logits = self.forward(x_batch)
        return torch.argmax(logits, dim=-1)

Для тестирования сети мы подготовили для вас класс ScoreEvaluator с двумя полезными методами:
 - *predict_tags*: получает батч данных и трансформирует его в список из токенов и предсказанных тегов;
 - *eval_conll*: вычисляет метрики precision, recall и F1

In [157]:
from evaluation_ner import ScoreEvaluator

evaluator = ScoreEvaluator(
    token_to_idx=token_to_idx,
    idx_to_tag=idx_to_tag,
    idx_to_token=idx_to_token,
)

### Эксперименты

Задайте BiLSTMModel. Рекомендуем начать с параметров:
- *batch_size*: 32;
- начальное значение *learning_rate*: 0.01-0.001
- *dropout_zeroed_probability*: 0.5-0.7
- *embedding_dim*: 100-200
- *rnn_hidden_size*: 150-200

Проведите эксперименты на данных. Настраивайте параметры по валидационной выборке, не используя тестовую. Ваше цель — настроить сеть так, чтобы качество модели по F1 мере на валидационной и тестовой выборках было не меньше 0.35. При некотором усердии, можно достичь результата 0.45 по F1 на обоих датасетах.

Если сеть плохо обучается, попробуйте использовать следующие модификации:

    * используйте gradient clipping
    * ограничивайте норму эмбеддингов через параметр max_norm (сопоставляйте с значениями в клиппинге)
    * на каждой итерации уменьшайте learning rate (например, в 1.1 раз)
    * попробуйте вместо Adam другие оптимизаторы
    * используйте l2 регуляризацию
    * экспериментируйте с значением dropout

Сделайте выводы о качестве модели, переобучении, чувствительности архитектуры к выбору гиперпараметров. Оформите результаты экспериментов в виде мини-отчета (в этом же ipython notebook).

**<font color='red'>Задание. Проведите требуемые эксперименты.</font>**

In [158]:
######################################
######### YOUR CODE HERE #############
######################################

In [196]:
def train(num_epochs, model, criterion, optimizer, train_loader, val_loader, 
          evaluator, max_grad_norm=None, scheduler=None, device="cuda"):
    
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
    
        for tokens_batch, tags_batch in train_loader:
            tokens_batch = tokens_batch.to(device)
            tags_batch = tags_batch.to(device)
    
            optimizer.zero_grad()
            
            outputs = model(tokens_batch)
            logits_reshaped = outputs.view(-1, outputs.shape[-1])
            tags_reshaped = tags_batch.view(-1)
    
            loss = criterion(logits_reshaped, tags_reshaped)
            loss.backward()
            if max_grad_norm:
                torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
            optimizer.step()
    
            total_loss += loss.item()
    
        total_loss /= len(train_loader)
    
        model.eval()
        val_loss = 0
        with torch.no_grad():
            for tokens_batch, tags_batch in val_loader:
                tokens_batch = tokens_batch.to(device)
                tags_batch = tags_batch.to(device)
    
                outputs = model(tokens_batch)
                logits_reshaped = outputs.view(-1, outputs.shape[-1])
                tags_reshaped = tags_batch.view(-1)
    
                loss = loss_fn(logits_reshaped, tags_reshaped)
                val_loss += loss.item()

        val_loss /= len(val_loader)
        if scheduler:
            scheduler.step()
        print(f"Epoch {epoch + 1}: Train Loss: {total_loss:.4f}, Val Loss: {val_loss:.4f}")
        print(evaluator.eval_conll(model, val_loader))
        print()
    return model


In [190]:
device = "cuda"

model = BiLSTMModel(
        vocabulary_size=len(token_to_idx),
        tag_space_size=len(tag_to_idx),
        pad_token_idx=token_to_idx['<PAD>'],
        embedding_dim=150,
        lstm_hidden_size=200,
        dropout_zeroed_probability=0.5).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss(ignore_index=token_to_idx['<PAD>'])

In [191]:
num_epochs=100
model_trained = train(num_epochs, model, criterion, optimizer, train_loader, val_loader, evaluator)

Epoch 1: Train Loss: 1.4605, Val Loss: 0.3502
{'precision': 0, 'recall': 0.0, 'f1': 0, 'n_predicted_entities': 0, 'n_true_entities': 537}

Epoch 2: Train Loss: 0.3161, Val Loss: 0.2659
{'precision': 0, 'recall': 0.0, 'f1': 0, 'n_predicted_entities': 0, 'n_true_entities': 537}

Epoch 3: Train Loss: 0.2738, Val Loss: 0.2441
{'precision': 0, 'recall': 0.0, 'f1': 0, 'n_predicted_entities': 0, 'n_true_entities': 537}

Epoch 4: Train Loss: 0.2531, Val Loss: 0.2303
{'precision': 0, 'recall': 0.0, 'f1': 0, 'n_predicted_entities': 0, 'n_true_entities': 537}

Epoch 5: Train Loss: 0.2310, Val Loss: 0.2090
{'precision': 0, 'recall': 0.0, 'f1': 0, 'n_predicted_entities': 0, 'n_true_entities': 537}

Epoch 6: Train Loss: 0.2111, Val Loss: 0.1967
{'precision': 0, 'recall': 0.0, 'f1': 0, 'n_predicted_entities': 0, 'n_true_entities': 537}

Epoch 7: Train Loss: 0.1991, Val Loss: 0.1894
{'precision': 0, 'recall': 0.0, 'f1': 0, 'n_predicted_entities': 0, 'n_true_entities': 537}

Epoch 8: Train Loss: 0.1865

In [192]:
print(evaluator.eval_conll(model_trained, train_loader))
print(evaluator.eval_conll(model_trained, val_loader))
print(evaluator.eval_conll(model_trained, test_loader))

{'precision': 95.91286772962049, 'recall': 95.14368456226332, 'f1': 95.52672780138673, 'n_predicted_entities': 4453, 'n_true_entities': 4489}
{'precision': 27.358490566037737, 'recall': 27.001862197392924, 'f1': 27.179006560449864, 'n_predicted_entities': 530, 'n_true_entities': 537}
{'precision': 50.47923322683706, 'recall': 26.158940397350992, 'f1': 34.46019629225736, 'n_predicted_entities': 313, 'n_true_entities': 604}


In [193]:
device = "cuda"

model = BiLSTMModel(
        vocabulary_size=len(token_to_idx),
        tag_space_size=len(tag_to_idx),
        pad_token_idx=token_to_idx['<PAD>'],
        embedding_dim=150,
        lstm_hidden_size=200,
        dropout_zeroed_probability=0.5,
        max_norm=1.0).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss(ignore_index=token_to_idx['<PAD>'])

num_epochs=100
model_trained = train(num_epochs, model, criterion, optimizer, train_loader, val_loader, evaluator, max_grad_norm=1.0)

Epoch 1: Train Loss: 1.5693, Val Loss: 0.3094
{'precision': 0, 'recall': 0.0, 'f1': 0, 'n_predicted_entities': 0, 'n_true_entities': 537}

Epoch 2: Train Loss: 0.2981, Val Loss: 0.2579
{'precision': 0, 'recall': 0.0, 'f1': 0, 'n_predicted_entities': 0, 'n_true_entities': 537}

Epoch 3: Train Loss: 0.2719, Val Loss: 0.2461
{'precision': 0, 'recall': 0.0, 'f1': 0, 'n_predicted_entities': 0, 'n_true_entities': 537}

Epoch 4: Train Loss: 0.2431, Val Loss: 0.2142
{'precision': 0, 'recall': 0.0, 'f1': 0, 'n_predicted_entities': 0, 'n_true_entities': 537}

Epoch 5: Train Loss: 0.2010, Val Loss: 0.1930
{'precision': 0, 'recall': 0.0, 'f1': 0, 'n_predicted_entities': 0, 'n_true_entities': 537}

Epoch 6: Train Loss: 0.1666, Val Loss: 0.1745
{'precision': 0, 'recall': 0.0, 'f1': 0, 'n_predicted_entities': 0, 'n_true_entities': 537}

Epoch 7: Train Loss: 0.1396, Val Loss: 0.1769
{'precision': 9.243697478991596, 'recall': 2.0484171322160147, 'f1': 3.3536585365853653, 'n_predicted_entities': 119, 'n

In [194]:
print(evaluator.eval_conll(model_trained, train_loader))
print(evaluator.eval_conll(model_trained, val_loader))
print(evaluator.eval_conll(model_trained, test_loader))

{'precision': 93.98178991783256, 'recall': 94.27489418578747, 'f1': 94.12811387900355, 'n_predicted_entities': 4503, 'n_true_entities': 4489}
{'precision': 47.875354107648725, 'recall': 31.471135940409685, 'f1': 37.97752808988764, 'n_predicted_entities': 353, 'n_true_entities': 537}
{'precision': 51.64556962025316, 'recall': 33.77483443708609, 'f1': 40.84084084084084, 'n_predicted_entities': 395, 'n_true_entities': 604}


In [202]:
device = "cuda"

model = BiLSTMModel(
        vocabulary_size=len(token_to_idx),
        tag_space_size=len(tag_to_idx),
        pad_token_idx=token_to_idx['<PAD>'],
        embedding_dim=150,
        lstm_hidden_size=200,
        dropout_zeroed_probability=0.5,
        max_norm=1.0).to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-2)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=1/1.1)
criterion = nn.CrossEntropyLoss(ignore_index=token_to_idx['<PAD>'])

num_epochs=100
model_trained = train(num_epochs, model, criterion, optimizer, train_loader, val_loader, evaluator, max_grad_norm=1.0, scheduler=scheduler)

Epoch 1: Train Loss: 0.4687, Val Loss: 0.1793
{'precision': 0, 'recall': 0.0, 'f1': 0, 'n_predicted_entities': 0, 'n_true_entities': 537}

Epoch 2: Train Loss: 0.1475, Val Loss: 0.1670
{'precision': 12.385321100917432, 'recall': 5.027932960893855, 'f1': 7.152317880794702, 'n_predicted_entities': 218, 'n_true_entities': 537}

Epoch 3: Train Loss: 0.1022, Val Loss: 0.1700
{'precision': 22.22222222222222, 'recall': 14.525139664804469, 'f1': 17.567567567567565, 'n_predicted_entities': 351, 'n_true_entities': 537}

Epoch 4: Train Loss: 0.0734, Val Loss: 0.1679
{'precision': 40.3125, 'recall': 24.022346368715084, 'f1': 30.10501750291715, 'n_predicted_entities': 320, 'n_true_entities': 537}

Epoch 5: Train Loss: 0.0489, Val Loss: 0.1781
{'precision': 44.606413994169095, 'recall': 28.491620111731844, 'f1': 34.77272727272727, 'n_predicted_entities': 343, 'n_true_entities': 537}

Epoch 6: Train Loss: 0.0306, Val Loss: 0.1758
{'precision': 45.572916666666664, 'recall': 32.588454376163874, 'f1': 3

In [203]:
print(evaluator.eval_conll(model_trained, train_loader))
print(evaluator.eval_conll(model_trained, val_loader))
print(evaluator.eval_conll(model_trained, test_loader))

{'precision': 94.9508489722967, 'recall': 94.67587435954556, 'f1': 94.81316229782489, 'n_predicted_entities': 4476, 'n_true_entities': 4489}
{'precision': 47.30077120822622, 'recall': 34.26443202979516, 'f1': 39.740820734341256, 'n_predicted_entities': 389, 'n_true_entities': 537}
{'precision': 49.53917050691244, 'recall': 35.59602649006622, 'f1': 41.42581888246628, 'n_predicted_entities': 434, 'n_true_entities': 604}


In [204]:
torch.save(model_trained, "model_best.pth")

In [208]:
device = "cuda"

model = BiLSTMModel(
        vocabulary_size=len(token_to_idx),
        tag_space_size=len(tag_to_idx),
        pad_token_idx=token_to_idx['<PAD>'],
        embedding_dim=150,
        lstm_hidden_size=200,
        dropout_zeroed_probability=0.5,
        max_norm=1.0).to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=1e-2, weight_decay=0.1)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=1, gamma=1/1.1)
criterion = nn.CrossEntropyLoss(ignore_index=token_to_idx['<PAD>'])

num_epochs=100
model_trained = train(num_epochs, model, criterion, optimizer, train_loader, val_loader, evaluator, max_grad_norm=1.0, scheduler=scheduler)

Epoch 1: Train Loss: 0.4805, Val Loss: 0.1985
{'precision': 0, 'recall': 0.0, 'f1': 0, 'n_predicted_entities': 0, 'n_true_entities': 537}

Epoch 2: Train Loss: 0.1543, Val Loss: 0.1638
{'precision': 9.620991253644315, 'recall': 6.145251396648045, 'f1': 7.500000000000001, 'n_predicted_entities': 343, 'n_true_entities': 537}

Epoch 3: Train Loss: 0.1005, Val Loss: 0.1667
{'precision': 19.672131147540984, 'recall': 13.40782122905028, 'f1': 15.946843853820598, 'n_predicted_entities': 366, 'n_true_entities': 537}

Epoch 4: Train Loss: 0.0747, Val Loss: 0.1676
{'precision': 32.25806451612903, 'recall': 20.484171322160147, 'f1': 25.056947608200456, 'n_predicted_entities': 341, 'n_true_entities': 537}

Epoch 5: Train Loss: 0.0496, Val Loss: 0.1797
{'precision': 48.12499999999999, 'recall': 28.67783985102421, 'f1': 35.93932322053676, 'n_predicted_entities': 320, 'n_true_entities': 537}

Epoch 6: Train Loss: 0.0298, Val Loss: 0.1735
{'precision': 43.17617866004963, 'recall': 32.402234636871505, 

In [209]:
print(evaluator.eval_conll(model_trained, train_loader))
print(evaluator.eval_conll(model_trained, val_loader))
print(evaluator.eval_conll(model_trained, test_loader))

{'precision': 94.83798882681565, 'recall': 94.5422143016262, 'f1': 94.68987059348507, 'n_predicted_entities': 4475, 'n_true_entities': 4489}
{'precision': 50.142450142450144, 'recall': 32.774674115456236, 'f1': 39.63963963963964, 'n_predicted_entities': 351, 'n_true_entities': 537}
{'precision': 49.00221729490022, 'recall': 36.58940397350993, 'f1': 41.89573459715639, 'n_predicted_entities': 451, 'n_true_entities': 604}


In [210]:
torch.save(model_trained, "model_best.pth")

Как можно заметить из экспериментов "базовая" модель без всяких трюков показывает довольно плохое качество. Добавление только градиентного клиппинга и ограничения нормы эмбеддингов позволяет увеличить f меру на валидации аж на 10 пунктов, а на тесте на 16. Смена оптимизатора на AdamW и добавление шедулера с уменьшением lr в 1.1 раз каждую эпуху позволяет ещё сильнее улучшить качество работы алгоритма. Настройка регуляризации позволяет слегка улучшить работу алгоритма, уменьшив его переобучение.

## Необучаемый пост-процессинг результата (4 балла).

Для обучения нейросетевой модели разметки используется поэлементная кросс-энтропия. При использовании на этапе инференса функции $\arg \max$ для получения выходной последовательности, мы не можем гарантировать согласованность предсказаний. Для согласованности необходимо вместо $\arg \max$ использовать другие функции получения предсказаний.

В модели CRF для получения предсказаний используется алгоритм Витерби. Напомним, что модель CRF моделирует вероятность последовательности $y$ при условии $x$ линейной моделью с вектором весов $w \in \mathbb{R}^d$, которая после некоторых преобразований записывается следующим образом:
$$
p(y|x, w) = \frac{1}{Z(x, w)} \exp\left( \sum_{i=1}^n \sum_{j = 1}^d w_j f_j(y_{i-1}, y_i, x_i, i) \right) =  \frac{1}{Z(x, w)} \exp\left( \sum_{i=1}^n G_{x, i}[y_{i-1}, y_i] \right)
$$

Модель необучаемого пост-процессинга **подробно описана** в приложении к заданию. Она сводится к следующим шагам.

1. Реализовать модель CRF с двумя признаками:
    
    * Лог-софтмакс выходов модели (выход, соответствующий $y_i$ тэгу для i-го токена будем обозначать $S_{i,y_i}$)    
    
    $$
    f_1(y_{i-1}, y_i, x_i, i) = S_{i,y_i}
    $$
    
    * Логарифмы вероятностей переходов

    $$
    f_2(y_{i-1}, y_i, x_i, i) = \log A[v=y_{i}, u=y_{i-1}] \mathbb{I}[i > 1] + \log C[v = y_i] \mathbb{I}[i = 1], \quad \text{где:}
    $$

    $$A_{vu} = \frac{\sum_{y}\sum_{i=2}^{|y|} \mathbb{I}[y_{i} = v, y_{i - 1} = u]}{\sum_{y}\sum_{i=2}^{|y|} \mathbb{I}[y_{i-1} = u]}
    $$
    $$
    C_v = \frac{\sum_{y}\mathbb{I}[y_{1} = v]}{\sum_{y}1}
    $$
    
2. Реализовать процедуру получения оптимальной выходной последовательности, используя алгоритм Витерби

3. Подобрать на валидационной выборке веса модели $w_1$ и $w_2$

Для исходной модели, дающей на валидационной и тестовой выборке F1 меру 0.408 и 0.46 соответственно, качество после такого пост-процессинга выросло до 0.461 и 0.493. Заметим, что для тестирования модели не нужно переобучать исходную модель. Для более устойчивого поведения модели, используйте сглаживание матрицы $A$ (добавьте перед нормировкой ко всем значениям одинаковое небольшое число).

**<font color='red'>Задание. Реализуйте требуемую модель, добейтесь улучшения качества на валидации и тесте, сделайте выводы.</font>**

In [330]:
class ViterbiPostprocesser:
    def __init__(self, model, smoothing=1.0, w=1.0):
        """
        model : torch.nn.Module
            Tagging model
        smoothing : float, constant in add-k-smoothing
        w : feature weight
             Use w for first feature weight and (1 - w) for second feature.
        """
        self.model = model
        self.smoothing = smoothing
        self.w = w

    def fit(self, dataset):
        """
        Fit the model using maximum likelihood method.

        dataset: torch.dataset
            One element if pair (sentence, tags)
        """
        ######################################
        ######### YOUR CODE HERE #############
        ######################################
        tag_cnt, tag_prev_next = {}, {}
        for sentence, tags in dataset:
            prev_tag = None
            for tag in tags:
                tag = tag.item()
                if tag not in tag_cnt:
                    tag_cnt[tag] = 0
                tag_cnt[tag] += 1
                
                if (prev_tag, tag) not in tag_prev_next:
                    tag_prev_next[(prev_tag, tag)] = 0
                tag_prev_next[(prev_tag, tag)] += 1
                prev_tag = tag

        self.A = torch.zeros((len(tag_cnt), len(tag_cnt)))
        self.C = torch.zeros(len(tag_cnt))
        tag_to_idx = {tag: i for i, tag in enumerate(tag_cnt.keys())}
        
        for (prev_tag, tag), count in tag_prev_next.items():
            if prev_tag:
                self.A[tag_to_idx[prev_tag], tag_to_idx[tag]] = count
            else:
                self.C[tag_to_idx[tag]] = count
        
        self.A = torch.log((self.A + 1e-7)) - torch.log((self.A.sum(axis=1, keepdims=True) + 1e-7))
        self.C = torch.log((self.C + 1e-7)) - torch.log((self.C.sum() + 1e-7))

    def decode(self, model_logprobs):
        """
        Viterbi decoding for input model output

        model_logprobs : torch.tensor
            Shape is (sequence_length, tag_space_size)
        """
        ######################################
        ######### YOUR CODE HERE #############
        ######################################
        sequence_length, tag_space_size = model_logprobs.shape
        
        prob_max = torch.zeros((sequence_length, tag_space_size))
        prev_max = torch.zeros((sequence_length, tag_space_size), dtype=int)
        
        prob_max[0, :] = self.w*model_logprobs[0, :] + (1 - self.w)*self.C
        for i in range(1, sequence_length):
            for j in range(tag_space_size):
                scores = self.w*model_logprobs[i, j] + (1 - self.w)*(prob_max[i-1] + self.A[:, j])
                prob_max[i, j] = torch.max(scores)
                prev_max[i, j] = torch.argmax(scores)
        
        y_pred = torch.zeros(sequence_length, dtype=int)
        y_pred[-1] = torch.argmax(prob_max[-1, :])
        
        for i in reversed(range(sequence_length-1)):
            y_pred[i] = prev_max[i+1, y_pred[i+1]]
        
        return y_pred

    def predict_for_batch(self, x_batch, device="cuda"):
        """
        Returns predictions for x_batch. Use viterbi decoding.

        return type: torch.LongTensor
        return shape: (number of samples in batch, number of words in sentence).

        ----------
        Parameters
        x_batch: torch.LongTensor with shape (number of samples in batch, number of words in sentence).
        """
        ######################################
        ######### YOUR CODE HERE #############
        ######################################
        outputs = self.model(x_batch.to(device)).cpu()
        y_pred = []
        for output in outputs:
            y_pred.append(self.decode(output))
        return torch.stack(y_pred)

Место для ваших экспериментов:

In [337]:
    ######################################
    ######### YOUR CODE HERE #############
    ######################################

In [336]:
viterbi = ViterbiPostprocesser(model_trained, w=0.95)
viterbi.fit(train_dataset)

print(evaluator.eval_conll(viterbi, val_loader))
print(evaluator.eval_conll(viterbi, test_loader))

{'precision': 55.83596214511041, 'recall': 32.960893854748605, 'f1': 41.4519906323185, 'n_predicted_entities': 317, 'n_true_entities': 537}
{'precision': 57.59162303664922, 'recall': 36.423841059602644, 'f1': 44.62474645030425, 'n_predicted_entities': 382, 'n_true_entities': 604}


In [332]:
viterbi = ViterbiPostprocesser(model_trained, w=0.85)
viterbi.fit(train_dataset)

print(evaluator.eval_conll(viterbi, val_loader))
print(evaluator.eval_conll(viterbi, test_loader))

{'precision': 57.87781350482315, 'recall': 33.5195530726257, 'f1': 42.45283018867925, 'n_predicted_entities': 311, 'n_true_entities': 537}
{'precision': 58.93333333333333, 'recall': 36.58940397350993, 'f1': 45.14811031664965, 'n_predicted_entities': 375, 'n_true_entities': 604}


In [334]:
viterbi = ViterbiPostprocesser(model_trained, w=0.5)
viterbi.fit(train_dataset)

print(evaluator.eval_conll(viterbi, val_loader))
print(evaluator.eval_conll(viterbi, test_loader))

{'precision': 52.280701754385966, 'recall': 27.746741154562383, 'f1': 36.25304136253042, 'n_predicted_entities': 285, 'n_true_entities': 537}
{'precision': 56.49717514124294, 'recall': 33.11258278145695, 'f1': 41.75365344467641, 'n_predicted_entities': 354, 'n_true_entities': 604}


In [335]:
viterbi = ViterbiPostprocesser(model_trained, w=0.3)
viterbi.fit(train_dataset)

print(evaluator.eval_conll(viterbi, val_loader))
print(evaluator.eval_conll(viterbi, test_loader))

{'precision': 48.63813229571984, 'recall': 23.277467411545622, 'f1': 31.48614609571788, 'n_predicted_entities': 257, 'n_true_entities': 537}
{'precision': 56.32911392405063, 'recall': 29.47019867549669, 'f1': 38.69565217391305, 'n_predicted_entities': 316, 'n_true_entities': 604}


Как можно заметить из экспериментов, данный метод генерации позволяет ещё сильнее улучшить качество работы модели. Однако необходимо корректно подбирать гиперпараметры. Сильно высокий показатель w или наоборот сильно низкий может даже ухудшить базовый алгоритм.

## Бонусная часть. Обучаемый постпроцессинг через CRF (2 балла).

Реализуйте сами / модифицируйте открытую реализацию CRF, соответствующую реализации обучаемого пост-процессинга в лекции. Например, можно использовать эту реализацию:
https://pytorch.org/tutorials/beginner/nlp/advanced_tutorial.html

В такой модели должно использоваться два типа признаков:

* $|Y|\times|Y|$ признаков, учитывающих выход модели:
$$	\phi_{uv}(y_{i-1}, y_i, x_i) = \mathbb{I}[y_{i-1} = u]\mathbb{I}[y_i = v] S_{i,y_i}$$
* $|Y|\times|Y|$ признаков, учитывающих связь меток:
$$ \psi_{uv}(y_{i-1}, y_i) = \mathbb{I}[y_{i-1} = u]\mathbb{I}[y_i = v] $$

Итоговое выражение для $G_{x, i}[y_{i-1}, y_i]$ выглядит так:
$$
G_{x, i}[y_{i-1}, y_i] = w(y_{i-1}, y_i) S_{i, y_i} + a(y_{i-1}, y_i)
$$


**<font color='red'>Задание. Обучите модель BiLSTM-CRF в едином пайплайне, добейтесь улучшения качества модели, сделайте выводы по проделанным экспериментам.</font>**

In [ ]:
######################################
######### YOUR CODE HERE #############
######################################

## Бонусная часть. Дополнительный char-LSTM слой (1 балл).

Добавьте к слою представлений слов обучаемые char-based представления. Каждое слово разделяется на символы и прогоняется через char-based сеть (например, через LSTM). Финальное состояние сети (или конкатенация двух финальных состояний в случае bidirectional LSTM) подаётся как дополнительное представление.

**<font color='red'> Задание. Обучите модель с дополнительным представлением и сравните качество с исходной моделью. Сделайте выводы по проделанным экспериментам.</font>**

In [ ]:
######################################
######### YOUR CODE HERE #############
######################################